In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mamba_ssm import Mamba
import numpy as np
import scipy.io

# Read the characteristic data of HSI
mat_file_path = r'.../.mat'

mat_data = scipy.io.loadmat(mat_file_path)

variable_name = list(mat_data.keys())[-1]

data = mat_data[variable_name]

print(f"The dimensions of X is: {data.shape}")
X = data


# Read the label data of HSI
mat_file_path = r'.../.mat'


mat_data = scipy.io.loadmat(mat_file_path)

variable_name = list(mat_data.keys())[-1]  # 获取最后一个键名，通常是我们需要的变量名

data = mat_data[variable_name]

print(f"The dimensions of Y is: {data.shape}")
Y = data

In [ ]:
# Convert HSI to two-dimensional data
X = X.reshape(X.shape[0] * X.shape[1], X.shape[2])
Y = Y.reshape(X.shape[0] * X.shape[1], X.shape[2])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np

# Converts a label to a 1-dimensional array
Y_pooled_reshaped = Y.ravel()

# A random forest classifier was used to evaluate the importance of features
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X, Y_pooled_reshaped)

# Acquired feature importance
importances = clf.feature_importances_

# Use SelectFromModel to select features that are above average in importance
threshold = np.mean(importances)
sfm = SelectFromModel(clf, threshold=threshold)

# Fits the data to a selector to transform the data set
X_important = sfm.fit_transform(X, Y_pooled_reshaped)

In [ ]:
from collections import Counter
X = X_important
y = Y_pooled_reshaped
X.shape
print(Counter(y))

In [ ]:
# Delete samples of class 0 in y and record the number of rows for these samples
rows_to_delete = []
for index, label in enumerate(y):
    if label == 0.0:
        rows_to_delete.append(index)

# Delete the sample of class 0 in y
y_filtered = np.delete(y, rows_to_delete, axis=0)

# Delete the corresponding row in X
X_filtered = np.delete(X, rows_to_delete, axis=0)

In [ ]:
# Data preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)

# Convert data to PyTorch tensor
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_filtered, dtype=torch.long)

# print(Counter(y_tensor))  # 对于训练集的标签

In [ ]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from collections import Counter

# Partition data set
train_size = int(0.7 * len(X_tensor))
val_size = int(0.2 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])


X_train, y_train = train_dataset.dataset.tensors
X_train, y_train = X_train.numpy(), y_train.numpy()

# Apply SMOTE only to the training set
smote = SMOTE(k_neighbors=2)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


# Converts the processed data back to the PyTorch tensor
X_train_tensor_smote = torch.tensor(X_train_smote, dtype=torch.float32)
y_train_tensor_smote = torch.tensor(y_train_smote, dtype=torch.long)

# Create a new TensorDataset
train_dataset_smote = TensorDataset(X_train_tensor_smote, y_train_tensor_smote)

# Create a DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset_smote, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class CustomMamba(Mamba):
    def __init__(self, d_model, d_state, d_conv, expand, lstm_hidden_size, lstm_num_layers):
        super(CustomMamba, self).__init__(d_model, d_state, d_conv, expand)
        self.lstm = nn.LSTM(
            input_size=d_model,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            batch_first=True,
            bidirectional=True  # Enable bidirectional
        )

        self.linear = nn.Linear(2*lstm_hidden_size, d_model)

    def forward(self, x):
        x, _ = self.lstm(x)

        x = self.linear(x)

        x = super(CustomMamba, self).forward(x)
        return x

lstm_hidden_size = 90
lstm_num_layers = 2

dim = X_tensor.shape[1]
model2 = CustomMamba(
    d_model=dim,
    d_state=16,
    d_conv=4,
    expand=2,
    lstm_hidden_size=lstm_hidden_size,
    lstm_num_layers=lstm_num_layers
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)

In [ ]:
num_epochs = 200
for epoch in range(num_epochs):
    model2.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to("cuda"), labels.to("cuda")

        # Forward propagation
        inputs = inputs.unsqueeze(1)
        outputs = model2(inputs)
        loss = criterion(outputs.squeeze(1), labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate and output the accuracy of each epoch
    model2.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to("cuda"), labels.to("cuda")

            inputs = inputs.unsqueeze(1)
            outputs = model2(inputs)
            _, predicted = torch.max(outputs.squeeze(1).data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total}%')

    model2.train()

In [ ]:
from sklearn.metrics import accuracy_score, cohen_kappa_score

model2.eval()
# Initializes two lists to store the predicted and true values
predicted_list = []
true_list = []
with torch.no_grad():
    correct = 0
    total = 0
    losses = []  # Used to store losses for each sample
    for inputs, labels in val_loader:
        inputs, labels = inputs.to("cuda"), labels.to("cuda")

        inputs = inputs.unsqueeze(1)
        outputs = model2(inputs)
        _, predicted = torch.max(outputs.squeeze(1).data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        losses.append(criterion(outputs.squeeze(1), labels).item())

        # Add the predicted and true values to the list
        predicted_list.extend(predicted.cpu().numpy())
        true_list.extend(labels.cpu().numpy())

    print(f'Accuracy of the network on the test samples: {correct / total}')

    kappa = cohen_kappa_score(true_list, predicted_list)
    print(f'Kappa coefficient on the test samples: {kappa:.4f}')

    avg_loss = sum(losses) / len(losses)
    print(f'Average loss on the test samples: {avg_loss:.4f}')

In [ ]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

model2.eval()

predicted_list = []
true_list = []
with torch.no_grad():
    correct = 0
    total = 0
    losses = []
    for inputs, labels in test_loader:
        inputs, labels = inputs.to("cuda"), labels.to("cuda")

        inputs = inputs.unsqueeze(1)
        outputs = model2(inputs)
        _, predicted = torch.max(outputs.squeeze(1).data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        losses.append(criterion(outputs.squeeze(1), labels).item())

        predicted_list.extend(predicted.cpu().numpy())
        true_list.extend(labels.cpu().numpy())


    print(f'Accuracy of the network on the test samples: {correct / total}')

    average_precision = precision_score(true_list, predicted_list, average='macro')
    print(f'Average Precision score on the test samples: {average_precision:.4f}')

    f1 = f1_score(true_list, predicted_list, average='macro')
    print(f'F1 score on the test samples: {f1:.4f}')

    kappa = cohen_kappa_score(true_list, predicted_list)
    print(f'Kappa coefficient on the test samples: {kappa:.4f}')

    avg_loss = sum(losses) / len(losses)
    print(f'Average loss on the test samples: {avg_loss:.4f}')

    conf_mat = confusion_matrix(true_list, predicted_list)

    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)

    plt.colorbar(cax)

    # Set the title and axis labels
    plt.title('Confusion matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    for (i, j), val in np.ndenumerate(conf_mat):
        if val > 40:
            text_color = 'white'
        else:
            text_color = 'black'
        ax.text(j, i, str(val), ha='center', va='center', color=text_color)

    plt.show()